In [1]:
import argparse
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score

In [2]:
use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [3]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="/home/z0m6ie/Documents/GitHub/datasets", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="/home/z0m6ie/Documents/GitHub/datasets", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))


np.random.seed(42)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=64, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=64, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="/home/z0m6ie/Documents/GitHub/datasets", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
# train_loader = torch.utils.data.DataLoader(
#     datasets.CIFAR10('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.RandomCrop(32, padding=4),
#                        transforms.RandomHorizontalFlip(),
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2))
#                    ])), batch_size=64, shuffle=True)
    

# test_loader = torch.utils.data.DataLoader(
#     datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     ])),
#     batch_size=1000, shuffle=True)

In [5]:
class Alex_Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Alex_Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return F.log_softmax(x)
    
def alex_net():
    model = Alex_Net()
    if use_cuda:
        model = model.cuda()
    return model

model = alex_net()
optimizer = optim.Adam(model.parameters())

print(model)

Alex_Net(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(5, 5))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d (64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d (192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d (384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Linear(in_features=256, out_features=10)
)


In [6]:
# Implement validation

def train(epoch):
    model.train()
    writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        loss = F.cross_entropy(output, target)
        loss.backward()
        accuracy = 100. * (correct/ len(output))
        optimizer.step()
        if batch_idx % 1 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0], accuracy))
            writer.add_scalar('Loss/Loss', loss.data[0], epoch)
            writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)

In [7]:
def validate(epoch):
    model.eval()
    writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy
    

In [8]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
# Tensor board data logger

In [10]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [11]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

for epoch in range(10):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
writer.export_scalars_to_json("./all_scalars.json")

writer.close()

/home/z0m6ie/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.304517, Accuracy: 9.38
Train Epoch: 0 [64/50000 (0%)]	Loss: 2.296066, Accuracy: 9.38
Train Epoch: 0 [128/50000 (0%)]	Loss: 2.283246, Accuracy: 17.19
Train Epoch: 0 [192/50000 (0%)]	Loss: 2.283081, Accuracy: 10.94
Train Epoch: 0 [256/50000 (1%)]	Loss: 2.256607, Accuracy: 10.94
Train Epoch: 0 [320/50000 (1%)]	Loss: 2.231259, Accuracy: 12.50
Train Epoch: 0 [384/50000 (1%)]	Loss: 2.403435, Accuracy: 1.56
Train Epoch: 0 [448/50000 (1%)]	Loss: 2.253029, Accuracy: 6.25
Train Epoch: 0 [512/50000 (1%)]	Loss: 2.262016, Accuracy: 14.06
Train Epoch: 0 [576/50000 (1%)]	Loss: 2.293967, Accuracy: 14.06
Train Epoch: 0 [640/50000 (1%)]	Loss: 2.257433, Accuracy: 17.19
Train Epoch: 0 [704/50000 (2%)]	Loss: 2.308920, Accuracy: 6.25
Train Epoch: 0 [768/50000 (2%)]	Loss: 2.303760, Accuracy: 7.81
Train Epoch: 0 [832/50000 (2%)]	Loss: 2.210677, Accuracy: 12.50
Train Epoch: 0 [896/50000 (2%)]	Loss: 2.242170, Accuracy: 17.19
Train Epoch: 0 [960/50000 (2%)]	Loss: 2.258165, A

Train Epoch: 0 [8128/50000 (18%)]	Loss: 1.893143, Accuracy: 32.81
Train Epoch: 0 [8192/50000 (18%)]	Loss: 1.861378, Accuracy: 21.88
Train Epoch: 0 [8256/50000 (18%)]	Loss: 1.955280, Accuracy: 28.12
Train Epoch: 0 [8320/50000 (18%)]	Loss: 1.900652, Accuracy: 21.88
Train Epoch: 0 [8384/50000 (19%)]	Loss: 1.874197, Accuracy: 34.38
Train Epoch: 0 [8448/50000 (19%)]	Loss: 1.950790, Accuracy: 26.56
Train Epoch: 0 [8512/50000 (19%)]	Loss: 1.881744, Accuracy: 25.00
Train Epoch: 0 [8576/50000 (19%)]	Loss: 1.866516, Accuracy: 29.69
Train Epoch: 0 [8640/50000 (19%)]	Loss: 1.737870, Accuracy: 29.69
Train Epoch: 0 [8704/50000 (19%)]	Loss: 1.923741, Accuracy: 20.31
Train Epoch: 0 [8768/50000 (19%)]	Loss: 1.875476, Accuracy: 23.44
Train Epoch: 0 [8832/50000 (20%)]	Loss: 2.035508, Accuracy: 21.88
Train Epoch: 0 [8896/50000 (20%)]	Loss: 1.975689, Accuracy: 23.44
Train Epoch: 0 [8960/50000 (20%)]	Loss: 1.708413, Accuracy: 31.25
Train Epoch: 0 [9024/50000 (20%)]	Loss: 1.796444, Accuracy: 35.94
Train Epoc

Train Epoch: 0 [16064/50000 (36%)]	Loss: 1.835534, Accuracy: 29.69
Train Epoch: 0 [16128/50000 (36%)]	Loss: 1.777076, Accuracy: 26.56
Train Epoch: 0 [16192/50000 (36%)]	Loss: 1.774252, Accuracy: 23.44
Train Epoch: 0 [16256/50000 (36%)]	Loss: 1.847236, Accuracy: 29.69
Train Epoch: 0 [16320/50000 (36%)]	Loss: 2.001331, Accuracy: 23.44
Train Epoch: 0 [16384/50000 (36%)]	Loss: 1.871632, Accuracy: 32.81
Train Epoch: 0 [16448/50000 (37%)]	Loss: 1.779956, Accuracy: 31.25
Train Epoch: 0 [16512/50000 (37%)]	Loss: 1.681905, Accuracy: 43.75
Train Epoch: 0 [16576/50000 (37%)]	Loss: 1.891678, Accuracy: 23.44
Train Epoch: 0 [16640/50000 (37%)]	Loss: 1.975493, Accuracy: 21.88
Train Epoch: 0 [16704/50000 (37%)]	Loss: 1.819325, Accuracy: 26.56
Train Epoch: 0 [16768/50000 (37%)]	Loss: 1.614969, Accuracy: 40.62
Train Epoch: 0 [16832/50000 (37%)]	Loss: 1.667786, Accuracy: 39.06
Train Epoch: 0 [16896/50000 (38%)]	Loss: 1.719475, Accuracy: 35.94
Train Epoch: 0 [16960/50000 (38%)]	Loss: 1.881917, Accuracy: 2

Train Epoch: 0 [23936/50000 (53%)]	Loss: 1.755105, Accuracy: 34.38
Train Epoch: 0 [24000/50000 (53%)]	Loss: 1.754218, Accuracy: 40.62
Train Epoch: 0 [24064/50000 (53%)]	Loss: 1.694733, Accuracy: 34.38
Train Epoch: 0 [24128/50000 (54%)]	Loss: 1.630515, Accuracy: 35.94
Train Epoch: 0 [24192/50000 (54%)]	Loss: 1.775435, Accuracy: 25.00
Train Epoch: 0 [24256/50000 (54%)]	Loss: 1.610011, Accuracy: 40.62
Train Epoch: 0 [24320/50000 (54%)]	Loss: 1.866371, Accuracy: 28.12
Train Epoch: 0 [24384/50000 (54%)]	Loss: 1.793886, Accuracy: 34.38
Train Epoch: 0 [24448/50000 (54%)]	Loss: 1.694623, Accuracy: 37.50
Train Epoch: 0 [24512/50000 (54%)]	Loss: 1.818243, Accuracy: 32.81
Train Epoch: 0 [24576/50000 (55%)]	Loss: 1.754140, Accuracy: 32.81
Train Epoch: 0 [24640/50000 (55%)]	Loss: 1.748742, Accuracy: 28.12
Train Epoch: 0 [24704/50000 (55%)]	Loss: 1.727049, Accuracy: 35.94
Train Epoch: 0 [24768/50000 (55%)]	Loss: 1.647327, Accuracy: 37.50
Train Epoch: 0 [24832/50000 (55%)]	Loss: 1.681043, Accuracy: 4

...

Train Epoch: 9 [21120/50000 (47%)]	Loss: 0.976442, Accuracy: 64.06
Train Epoch: 9 [21184/50000 (47%)]	Loss: 1.200057, Accuracy: 59.38
Train Epoch: 9 [21248/50000 (47%)]	Loss: 0.886175, Accuracy: 76.56
Train Epoch: 9 [21312/50000 (47%)]	Loss: 0.898398, Accuracy: 60.94
Train Epoch: 9 [21376/50000 (47%)]	Loss: 1.011885, Accuracy: 65.62
Train Epoch: 9 [21440/50000 (48%)]	Loss: 1.117040, Accuracy: 60.94
Train Epoch: 9 [21504/50000 (48%)]	Loss: 1.042046, Accuracy: 56.25
Train Epoch: 9 [21568/50000 (48%)]	Loss: 1.153263, Accuracy: 60.94
Train Epoch: 9 [21632/50000 (48%)]	Loss: 1.014260, Accuracy: 57.81
Train Epoch: 9 [21696/50000 (48%)]	Loss: 1.109845, Accuracy: 62.50
Train Epoch: 9 [21760/50000 (48%)]	Loss: 0.848428, Accuracy: 73.44
Train Epoch: 9 [21824/50000 (48%)]	Loss: 1.504017, Accuracy: 56.25
Train Epoch: 9 [21888/50000 (49%)]	Loss: 0.996656, Accuracy: 64.06
Train Epoch: 9 [21952/50000 (49%)]	Loss: 1.189231, Accuracy: 60.94
Train Epoch: 9 [22016/50000 (49%)]	Loss: 1.035564, Accuracy: 6

Train Epoch: 9 [28992/50000 (64%)]	Loss: 0.839596, Accuracy: 67.19
Train Epoch: 9 [29056/50000 (64%)]	Loss: 1.022736, Accuracy: 67.19
Train Epoch: 9 [29120/50000 (65%)]	Loss: 1.286224, Accuracy: 48.44
Train Epoch: 9 [29184/50000 (65%)]	Loss: 0.844685, Accuracy: 70.31
Train Epoch: 9 [29248/50000 (65%)]	Loss: 0.945310, Accuracy: 64.06
Train Epoch: 9 [29312/50000 (65%)]	Loss: 0.899506, Accuracy: 75.00
Train Epoch: 9 [29376/50000 (65%)]	Loss: 1.002925, Accuracy: 62.50
Train Epoch: 9 [29440/50000 (65%)]	Loss: 1.007193, Accuracy: 64.06
Train Epoch: 9 [29504/50000 (65%)]	Loss: 0.943311, Accuracy: 68.75
Train Epoch: 9 [29568/50000 (66%)]	Loss: 1.236746, Accuracy: 51.56
Train Epoch: 9 [29632/50000 (66%)]	Loss: 0.990319, Accuracy: 67.19
Train Epoch: 9 [29696/50000 (66%)]	Loss: 1.165577, Accuracy: 57.81
Train Epoch: 9 [29760/50000 (66%)]	Loss: 1.136169, Accuracy: 57.81
Train Epoch: 9 [29824/50000 (66%)]	Loss: 1.140717, Accuracy: 60.94
Train Epoch: 9 [29888/50000 (66%)]	Loss: 0.997038, Accuracy: 6

Train Epoch: 9 [36864/50000 (82%)]	Loss: 1.342088, Accuracy: 59.38
Train Epoch: 9 [36928/50000 (82%)]	Loss: 1.020514, Accuracy: 67.19
Train Epoch: 9 [36992/50000 (82%)]	Loss: 0.900388, Accuracy: 70.31
Train Epoch: 9 [37056/50000 (82%)]	Loss: 1.066914, Accuracy: 53.12
Train Epoch: 9 [37120/50000 (82%)]	Loss: 1.081337, Accuracy: 68.75
Train Epoch: 9 [37184/50000 (83%)]	Loss: 1.236254, Accuracy: 51.56
Train Epoch: 9 [37248/50000 (83%)]	Loss: 0.884920, Accuracy: 68.75
Train Epoch: 9 [37312/50000 (83%)]	Loss: 0.962068, Accuracy: 62.50
Train Epoch: 9 [37376/50000 (83%)]	Loss: 0.820030, Accuracy: 68.75
Train Epoch: 9 [37440/50000 (83%)]	Loss: 1.283333, Accuracy: 59.38
Train Epoch: 9 [37504/50000 (83%)]	Loss: 1.269830, Accuracy: 60.94
Train Epoch: 9 [37568/50000 (83%)]	Loss: 1.138543, Accuracy: 64.06
Train Epoch: 9 [37632/50000 (84%)]	Loss: 0.982419, Accuracy: 71.88
Train Epoch: 9 [37696/50000 (84%)]	Loss: 1.057850, Accuracy: 67.19
Train Epoch: 9 [37760/50000 (84%)]	Loss: 0.920725, Accuracy: 6

Train Epoch: 9 [44736/50000 (99%)]	Loss: 0.946480, Accuracy: 65.62
Train Epoch: 9 [44800/50000 (99%)]	Loss: 0.985609, Accuracy: 71.88
Train Epoch: 9 [44864/50000 (100%)]	Loss: 0.998645, Accuracy: 65.62
Train Epoch: 9 [44928/50000 (100%)]	Loss: 0.853477, Accuracy: 75.00
Train Epoch: 9 [5624/50000 (100%)]	Loss: 1.086629, Accuracy: 62.50

Validation set: Average loss: 1.0467, Accuracy: 3167/5000 (63.34%)



NameError: name 'writer' is not defined

In [ ]:
test(epoch)